In [517]:
# Uploading files from your local file system

from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving datasets.zip to datasets (2).zip
User uploaded file "datasets.zip" with length 16245093 bytes


In [518]:
! unzip datasets.zip
! unzip brazilian-ecommerce

Archive:  datasets.zip
replace geojson/natal.geojson? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: geojson/natal.geojson   
  inflating: geojson/geojs-41-mun.json  
  inflating: geojson/.DS_Store       
  inflating: geojson/geojs-15-mun.json  
  inflating: geojson/geojs-14-mun.json  
  inflating: geojson/geojs-21-mun.json  
  inflating: geojson/geojs-12-mun.json  
  inflating: geojson/geojs-13-mun.json  
  inflating: geojson/geojs-26-mun.json  
  inflating: geojson/geojs-27-mun.json  
  inflating: geojson/geojs-51-mun.json  
  inflating: geojson/geojs-50-mun.json  
  inflating: geojson/geojs-31-mun.json  
  inflating: geojson/geojs-22-mun.json  
  inflating: geojson/geojs-23-mun.json  
  inflating: geojson/geojs-28-mun.json  
  inflating: geojson/geojs-29-mun.json  
  inflating: geojson/geojs-16-mun.json  
  inflating: geojson/geojs-17-mun.json  
  inflating: geojson/geojs-35-mun.json  
  inflating: geojson/geojs-42-mun.json  
  inflating: geojson/geojs-43-mun.json  
  inflatin

In [519]:
! pip install shapely folium unidecode

In [0]:
import os
import folium
import json
import pandas as pd
from branca.colormap import linear
import numpy as np
from numpy import random
import unidecode

In [0]:
geo = pd.read_csv("geolocation_olist_public_dataset.csv")
customers = pd.read_csv("olist_public_dataset_v2_customers.csv")
translation = pd.read_csv("product_category_name_translation.csv")
v2 = pd.read_csv("olist_public_dataset_v2.csv")
classif = pd.read_csv("olist_classified_public_dataset.csv")
payments = pd.read_csv("olist_public_dataset_v2_payments.csv")

In [0]:
rn_state = v2[v2['customer_state'] == 'RN']

In [0]:
compras_rn = {}
for _, row in rn_state.iterrows():
  if row['customer_city'] in compras_rn:
    compras_rn[row['customer_city']] += row['order_products_value']
  else:
    compras_rn[row['customer_city']] = row['order_products_value']

In [533]:
df = pd.DataFrame.from_dict(compras_rn, orient='index')
df.reset_index(level=0, inplace=True)
df.columns = ['municipios','gasto_em_reais']
df.head()

,municipios,gasto_em_reais
0,Mossoro,6630.31
1,Itau,241.80
2,Carnauba dos Dantas,148.99
3,Natal,30975.87
4,Caico,4023.11


In [0]:
RN = os.path.join('geojson','geojs-24-mun.json')
geo_json_data = json.load(open(RN,encoding='latin-1'))

In [535]:
colormap = linear.RdPu_03.scale(
    df.gasto_em_reais.min(),
    df.gasto_em_reais.max())

print(colormap(5000.0))

colormap

#fdcbd0


In [0]:
cidades_faltantes = ['Antonio Martins','Ares','Augusto Severo','Baia Formosa','Barauna','Barcelona','Bento Fernandes','Bom Jesus','Brejinho',
                    'Caicara do Norte', 'Caicara do Rio do Vento','Carnaubais', 'Coronel Ezequiel', 'Coronel Joao Pessoa', 'Doutor Severiano',
                    'Equador', 'Fernando Pedroza', 'Grossos', 'Itaja', 'Janduis', 'Januario Cicco', 'Japi', 'Jardim de Angicos', 'Joao Dias',
                    'Jundia', 'Lagoa d\'Anta', 'Lagoa de Pedras', 'Lagoa de Velhos', 'Lagoa Salgada', 'Lajes', 'Lajes Pintadas', 'Lucrecia',
                    'Luis Gomes', 'Major Sales', 'Marcelino Vieira', 'Maxaranguape', 'Monte Alegre', 'Monte das Gameleiras',
                    'Olho-d\'Agua do Borges', 'Parana', 'Parau', 'Parazinho', 'Passa e Fica', 'Passagem', 'Pedra Grande','Pedro Avelino',
                    'Pedro Velho','Porto do Mangue','Presidente Juscelino', 'Pureza', 'Rafael Fernandes', 'Rafael Godeiro', 'Riachuelo', 'Tibau',
                    'Ruy Barbosa','Sao Bento do Norte', 'Sao Bento do Trairi', 'Sao Francisco do Oeste','Sao Jose do Campestre',
                    'Sao Miguel do Gostoso','Sao Pedro','Senador Eloi de Souza','Senador Georgino Avelino','Serra Negra do Norte',
                    'Severiano Melo','Sitio Novo', 'Taboleiro Grande','Taipu','Tangara', 'Tenente Laurentino Cruz','Timbauba dos Batistas',
                    'Touros','Triunfo Potiguar','Umarizal', 'Varzea', 'Venha-Ver']

for i in cidades_faltantes:
  compras_rn[i] = 0

In [537]:
m = folium.Map(
    location=[-5.66556, -36.60111],
    zoom_start=8,
    tiles='Stamen Terrain'
)


folium.GeoJson(
    geo_json_data,
    name='Comparativo de Gastos em Reais com E-commerce entre Municípios do RN',
    style_function=lambda feature: {
        'fillColor': colormap(compras_rn[unidecode.unidecode(feature['properties']['description'])]),
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.9,
    }
).add_to(m)

colormap.caption = 'Gasto em Reais por Município do RN com E-Commerce'
colormap.add_to(m)

folium.LayerControl().add_to(m)

m